In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import pandas as pd
from IPython.display import clear_output

import quick_pp.las_handler as las
from quick_pp.lithology.carbonate import Carbonate
from quick_pp.plotter import plotly_log
from quick_pp.qaqc import badhole_flag, mask_outside_threshold, neu_den_xplot_hc_correction

In [ ]:
with open('x3.las', 'rb') as f:
    df, header, _ = las.read_las_file(f)

# Mask outside threshold
df = mask_outside_threshold(df, True)

# Flag bad hole
df = badhole_flag(df) if 'CALI' in df.columns else df
clear_output()

In [ ]:
fig = plotly_log(df)
fig.show()
# fig.write_html('plot.html')

In [ ]:
args = {
    # 'dry_clay_point': (0.27, 2.8)
    # 'silt_line_angle': 119
}
carb_model = Carbonate(**args)

clean_df = df[['GR', 'NPHI', 'RHOB', 'PEF', 'DEPTH']]  # .dropna()
vcld, vcalc, vdolo = carb_model.estimate_lithology(
    clean_df['GR'], clean_df['NPHI'], clean_df['RHOB'], clean_df['PEF'],
    model='single', method='neu_den',  # method='den_pef',  #  carbonate_type='dolomite'
)
df_carb = pd.DataFrame(
    {'VCLW': vcld, 'VCALC': vcalc, 'VDOLO': vdolo},
    index=clean_df.DEPTH
)
df_carb = df.merge(df_carb, how='left', on='DEPTH', suffixes=('_ORI', ''))

In [ ]:
fig = plotly_log(df_carb)
fig.show()
# fig.write_html('plot.html')

## Porosity

In [ ]:
from quick_pp.porosity import density_porosity, rho_matrix

In [ ]:
rho_ma = rho_matrix(vcalc=df_carb['VCALC'], vdolo=df_carb['VDOLO'], vclay=df_carb['VCLW'])

df_carb['PHID'] = density_porosity(df_carb['RHOB'], rho_ma)
df_carb.plot(x='DEPTH', y=['PHIT', 'PHID'], figsize=(20, 5))

## Hydrocarbon Correction

In [ ]:
# Correct NPHI and RHOB for HC
df_corr = df.copy()
nphihc, rhobhc = neu_den_xplot_hc_correction(df_corr['NPHI'], df_corr['RHOB'], gr=df_corr['GR'],
                                             dry_sand_point=(0.0, 2.71),
                                             dry_clay_point=(.33, 2.7),
                                             corr_angle=50)
df_corr[['NPHI', 'RHOB']] = pd.DataFrame({'NPHI': nphihc, 'RHOB': rhobhc}).astype(float)

clean_df = df_corr[['DEPTH', 'GR', 'NPHI', 'RHOB', 'PEF']]  # .dropna()
vcld, vcalc, vdolo = carb_model.estimate_lithology(
    clean_df['GR'], clean_df['NPHI'], clean_df['RHOB'], clean_df['PEF'],
    model='single', method='neu_den',  # method='neu_den', # carbonate_type='dolomite'
)
df_carb = pd.DataFrame(
    {'VCLW': vcld, 'VCALC': vcalc, 'VDOLO': vdolo},
    index=clean_df.DEPTH
)
df_carb = df.merge(df_carb, how='left', on='DEPTH', suffixes=('_ORI', ''))

In [ ]:
fig = plotly_log(df_carb)
fig.show()
# fig.write_html('plot.html')